In [12]:
import pandas as pd

df = pd.read_csv('food_list.csv',encoding='cp949')
data = df[['구분', '음식명']] # 필요한 데이터만 가져오기

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,1)) 
cv_category = cv.fit_transform(data['구분'])
cv.vocabulary_ # 구분 -> 숫자

{'어류': 24,
 '구이': 5,
 '젓갈': 30,
 '조림': 31,
 '채소류': 35,
 '나물': 9,
 '볶음': 17,
 '양념': 23,
 '과일류': 4,
 '서류': 19,
 '김치': 8,
 '한과': 40,
 '마른찬': 13,
 '갑각류': 0,
 '생채': 18,
 '음청류': 27,
 '고명': 2,
 '장아찌': 28,
 '육류': 26,
 '전골': 29,
 '청주': 36,
 '패류': 38,
 '해조류': 41,
 '만두': 14,
 '찌개': 34,
 '곡류': 3,
 '연체류': 25,
 '기타': 7,
 '견과류': 1,
 '편육': 39,
 '두부': 11,
 '떡국': 12,
 '숙채': 21,
 '두류': 10,
 '증류주': 33,
 '버섯류': 16,
 '수제비': 20,
 '순대': 22,
 '튀김': 37,
 '국수': 6,
 '족편': 32,
 '무침': 15}

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_category = cosine_similarity(cv_category, cv_category).argsort()[:,::-1]
print(similarity_category)
similarity_category.shape

[[750 246 255 ... 496 495   0]
 [750 246 255 ... 496 495   0]
 [750 246 255 ... 496 495   0]
 ...
 [750 246 255 ... 496 495   0]
 [750 246 255 ... 496 495   0]
 [750 246 255 ... 496 495   0]]


(751, 751)

In [18]:
def recommend_menu(df, menu_name, top=10):
	target_menu_idx = df[df['음식명'] == menu_name].index.values

	sim_idx = similarity_category[target_menu_idx, :top].reshape(-1)
	sim_idx = sim_idx[sim_idx != target_menu_idx]

	result = df.iloc[sim_idx]
	return result

In [25]:
recommend_menu(data, menu_name='가자미구이')

,구분,음식명
52,구이,고등어(소금)구이
410,구이,쇠고기소금구이
341,구이,북어구이
603,구이,조기구이
523,구이,옥돔구이
166,구이,대합구이
125,구이,꽁치구이
293,구이,민어구이
580,구이,장어구이
614,구이,주물럭구이
